In [1]:
import os
import json
import csv
from io import StringIO

from dotenv import load_dotenv

from s2ag_corpus.sql import CREATE_CITATIONS_TABLE_WITH_INDICES

In [2]:
load_dotenv()
base_dir = os.getenv("BASE_DIR")

In [3]:
from s2ag_corpus.database_catalogue import local_connection

connection = local_connection()

def drop_and_replace_citations_table():
    with connection.cursor() as cursor:
        cursor.execute('drop table if exists citations')
        cursor.execute(CREATE_CITATIONS_TABLE_WITH_INDICES)

In [4]:
drop_and_replace_citations_table()
    

In [5]:
def read_records_from_file(file_path):
    """A generator function that returns reformatted lines in a file."""
    output = StringIO()
    writer = csv.writer(output, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
    count = 0
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            jd = json.loads(line)
            record = (jd['citationid'],jd['citingcorpusid'],jd['citedcorpusid'],jd['isinfluential'])
            output.seek(0)
            output.truncate(0)
            writer.writerow(record)
            count += 1
            if count % 100000 == 0:
                print(count)
            yield output.getvalue()

In [6]:
class JsonFileInserter:
    def __init__(self, generator):
        self.generator = generator
        self.buffer = ''
        self.count = 0

    def read(self, size=-1):
        # Fill the buffer to meet the size requirement or if size is -1 then try to exhaust the generator
        while (size < 0 or len(self.buffer) < size) and (chunk := next(self.generator, None)) is not None:
            self.buffer += chunk
            if not self.buffer.endswith('\n'):
                self.buffer += '\n'  # Ensure each chunk ends with a newline

        if size < 0 or len(self.buffer) <= size:
            to_return, self.buffer = self.buffer, ''
        else:
            to_return, self.buffer = self.buffer[:size], self.buffer[size:]

        return to_return

In [7]:
def copy_json_to_papers(test_file):
    adapter = JsonFileInserter(read_records_from_file(test_file))
    with connection.cursor() as cursor:
        cursor.copy_from(adapter, 'citations', sep=',', null='')
        connection.commit()
    print('done')

In [8]:
test_file = base_dir+'/2024-04-02/citations/citations0'
copy_json_to_papers(test_file)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [9]:
connection.rollback()